In [3]:
import numpy as np
import random
import itertools
from operator import attrgetter
from collections import defaultdict
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd

import PIL
from jinja2 import Environment
from IPython.display import HTML, display, Math
import os

In [1]:
suits = {'Hearts', 'Clubs', 'Spades', 'Diamonds'}

card_name_map = {
            'Two': 2,
            'Three': 3,
            'Four': 4,
            'Five': 5,
            'Six': 6,
            'Seven': 7,
            'Eight': 8,
            'Nine': 9,
            'Ten': 10,
            'Jack': 11, 
            'Queen': 12, 
            'King': 13, 
            'Ace': 14
            }

class Card:
    def __init__(self, name, suit):
        self.suit = suit
        self.name = name
        if self.name == 'Six':
            self.name_plural = 'Sixes'
        else:
            self.name_plural = self.name + 's'
        self.value = card_name_map[self.name]
        if self.value < 10:
            self.abbreviation = str(self.value)
        elif self.value == 10:
            self.abbreviation = 'T'
        else:
            self.abbreviation = self.name[0]
    
    def __str__(self):
        return '{} of {}'.format(self.name, self.suit)
    
    def __eq__(self, other):
        return self.value == other.value

In [142]:
class Deck:
    def __init__(self):
        self.deck = []
        self.card_location_map = {}
        for suit in suits:
            for card_name in card_name_map:
                self.deck.append(Card(card_name, suit))
        self.shuffle()
                
    def __str__(self):
        return ''.join(['{} \n'.format(card) for card in self.deck])
    
    def assign_locations(self):
        self.card_location_map = {}
        for i, card in enumerate(self.deck):
            self.card_location_map[str(card)] = i
        
    
    def shuffle(self):
        random.shuffle(self.deck)
        self.assign_locations()
    
    def remove(self, card):
        card = self.deck.pop(self.card_location_map[str(card)])
        self.assign_locations()
        return card
    
    def draw(self):
        card = self.deck.pop(0)
        self.assign_locations()
        return card
    
    def __str__(self):
        return ''.join(['{} \n'.format(card) for card in self.deck])

In [209]:
class Player:
    def __init__(self, name, strategy):
        self.name = name
        self.attempts = 0
        self.drinks = 0
        self.cards = []
        self.strategy = strategy  # {'user', 'random', 'optimal'}
    
    def red_or_black(self, card, board_cards):
        if self.strategy == 'user':
            choice = input("Red or Black?").lower()
        elif self.strategy == 'random':
            fork = np.random.randint(2)
            if fork == 0:
                choice = 'red'
            else:
                choice = 'black'
        elif self.strategy == 'optimal':
            counts = {'red': 0, 'black': 0}
            for board_card in board_cards:
                if board_card.suit == 'Hearts' or board_card.suit == 'Diamonds':
                    counts['red'] += 1
                if board_card.suit == 'Clubs' or board_card.suit == 'Spades':
                    counts['black'] += 1
            
            if counts['red'] > counts['black']:
                choice = 'black'
            elif counts['red'] < counts['black']:
                choice = 'red'
            else:
                fork = np.random.randint(2)
                if fork == 0:
                    choice = 'red'
                else:
                    choice = 'black'
                
        self.cards.append(card)
#         print(card)
        if card.suit == 'Hearts' or card.suit == 'Diamonds':
            card_color = 'red'
        else:
            card_color = 'black'
        if choice != card_color:
            self.drinks += 1
    
    def high_or_low(self, card, board_cards):
        if self.strategy == 'user':
            choice = input("Higher or Lower?\n").lower()
        elif self.strategy == 'random':
            fork = np.random.randint(2)
            if fork == 0:
                choice = 'higher'
            else:
                choice = 'lower'
        elif self.strategy == 'optimal':
            counts = {'higher': 0, 'lower': 0}
            for board_card in board_cards:
                if board_card.value > card.value:
                    counts['higher'] += 1
                else:
                    counts['lower'] += 1
            
            possible_lower = ((card.value - 1) * 4) - 1 - counts['lower']
            possible_higher = ((14 - card.value) * 4) + 3 - counts['higher']
            
            if possible_lower > possible_higher:
                choice = 'higher'
            elif possible_lower < possible_higher:
                choice = 'lower'
            else:
                fork = np.random.randint(2)
                if fork == 0:
                    choice = 'higher'
                else:
                    choice = 'lower'

            
        self.cards.append(card)
#         print(card)
        if card.value > self.cards[0].value:
            higher = True
        else:
            higher = False
        
        if higher:
            if choice != "higher":
                self.drinks += 2
        
        else:
            if choice != "lower":
                self.drinks += 2
                
    def in_or_out(self, card, board_cards):
        max_card = max(self.cards, key=attrgetter('value'))
        min_card = min(self.cards, key=attrgetter('value'))
        inside = card.value in range(min_card.value, max_card.value + 1)
        
        if self.strategy == 'user':
            choice = input("Inside or Outside?\n").lower()
        elif self.strategy == 'random':
            fork = np.random.randint(2)
            if fork == 0:
                choice = 'inside'
            else:
                choice = 'outside'
        elif self.strategy == 'optimal':
            counts = {'inside': 0, 'outside': 0}
            for board_card in board_cards:
                if board_card.value in range(min_card.value, max_card.value + 1):
                    counts['inside'] += 1
                else:
                    counts['outside'] += 1
            possible_inside = 2 + abs(max_card.value-min_card.value) * 4
            possible_outside = 52 - possible_inside
            
            if possible_inside > possible_outside:
                choice = 'outside'
            elif possible_inside < possible_outside:
                choice = 'inside'
            else:
                fork = np.random.randint(2)
                if fork == 0:
                    choice = 'inside'
                else:
                    choice = 'outside'
        self.cards.append(card)
#         print(card)
        if inside:
            if choice != 'inside':
                self.drinks += 3
        else:
            if choice != 'outside':
                self.drinks += 3
    
    def suit(self, card, board_cards):
        if self.strategy == 'user':
            choice = input("Suit?\n").lower()
        elif self.strategy == 'random':
            fork = np.random.randint(4)
            if fork == 0:
                choice = 'hearts'
            elif fork == 1:
                choice = 'clubs'
            elif fork == 2:
                choice = 'diamonds'
            elif fork == 3:
                choice = 'spades'
        elif self.strategy == 'optimal':
            counts = {'Spades': 0, 'Clubs': 0, 'Hearts': 0, 'Diamonds': 0}
            for board_card in board_cards:
                counts[board_card.suit] += 1
            min_suit = min(counts.items(), key=lambda x: x[1])
            possible_min_suits = [min_suit]
            for suit in counts:
#                 print(suit, min_suit)
                if suit != min_suit and counts[suit] == counts[min_suit[0]]:
                    possible_min_suits.append(suit)
            
            if len(possible_min_suits) == 1:
                choice = min_suit.lower()
            else:
                fork = np.random.randint(len(possible_min_suits))
                choice = possible_min_suits[fork][0].lower()
            
        self.cards.append(card)
#         print(card)
        if card.suit.lower() != choice:
            self.drinks += 4
            
                

In [217]:
class Game:
    def __init__(self, players):
        self.players = players
        self.deck = Deck()
        self.board_cards = []
    
    def deal_round(self):
        for i in range(4):
            for player in self.players:
                card = self.deck.draw()
#                 print(player.name)
                if i == 0:
                    player.red_or_black(card, self.board_cards)
                if i == 1:
                    player.high_or_low(card, self.board_cards)
                if i == 2:
                    player.in_or_out(card, self.board_cards)
                if i == 3:
                    player.suit(card, self.board_cards)
                self.board_cards.append(card)
#                 print(player.drinks)
        return {player.name: player.drinks for player in enumerate(self.players)}
#         print([str(card) for card in self.board_cards])

#     def give_round(self):
    
#     def take_round(self):
        
#     def ride_the_bus(self, player)
        

# Experiments

- Random Player vs Optimal Player 10000 games
- Average number of drinks for each player
- Expect optimal player to have a lower average

In [219]:
random_average = 0
optimal_average = 0

players = [
    Player('p1', 'optimal'),
    Player('p2', 'optimal')
]

for i in range(10000):
#     g = Game([Player('p{}'.format(i), 'optimal') for i in range(5)])
    g = Game(players)
#     print(g.deal_round())
    result = g.deal_round()

    random_average += result[0]
    optimal_average += result[1]

print(random_average / 10000)
print(optimal_average / 10000)
    


AttributeError: 'tuple' object has no attribute 'drinks'

In [213]:
print(inside)

2


In [120]:
for i in range(2, 15):
    lower = ((i - 1) * 4) - 1
    higher = ((14 - i) * 4) + 3
    print(i, lower, higher)

2 3 51
3 7 47
4 11 43
5 15 39
6 19 35
7 23 31
8 27 27
9 31 23
10 35 19
11 39 15
12 43 11
13 47 7
14 51 3


In [131]:
for i in range(2, 15):
    for j in range(2, 15):
        low = i
        high = j
        inside = 2 + abs(low-high) * 4
        outside = 52 - inside
        print(low, high, inside, outside)

2 2 2 50
2 3 6 46
2 4 10 42
2 5 14 38
2 6 18 34
2 7 22 30
2 8 26 26
2 9 30 22
2 10 34 18
2 11 38 14
2 12 42 10
2 13 46 6
2 14 50 2
3 2 6 46
3 3 2 50
3 4 6 46
3 5 10 42
3 6 14 38
3 7 18 34
3 8 22 30
3 9 26 26
3 10 30 22
3 11 34 18
3 12 38 14
3 13 42 10
3 14 46 6
4 2 10 42
4 3 6 46
4 4 2 50
4 5 6 46
4 6 10 42
4 7 14 38
4 8 18 34
4 9 22 30
4 10 26 26
4 11 30 22
4 12 34 18
4 13 38 14
4 14 42 10
5 2 14 38
5 3 10 42
5 4 6 46
5 5 2 50
5 6 6 46
5 7 10 42
5 8 14 38
5 9 18 34
5 10 22 30
5 11 26 26
5 12 30 22
5 13 34 18
5 14 38 14
6 2 18 34
6 3 14 38
6 4 10 42
6 5 6 46
6 6 2 50
6 7 6 46
6 8 10 42
6 9 14 38
6 10 18 34
6 11 22 30
6 12 26 26
6 13 30 22
6 14 34 18
7 2 22 30
7 3 18 34
7 4 14 38
7 5 10 42
7 6 6 46
7 7 2 50
7 8 6 46
7 9 10 42
7 10 14 38
7 11 18 34
7 12 22 30
7 13 26 26
7 14 30 22
8 2 26 26
8 3 22 30
8 4 18 34
8 5 14 38
8 6 10 42
8 7 6 46
8 8 2 50
8 9 6 46
8 10 10 42
8 11 14 38
8 12 18 34
8 13 22 30
8 14 26 26
9 2 30 22
9 3 26 26
9 4 22 30
9 5 18 34
9 6 14 38
9 7 10 42
9 8 6 46
9 9 2 50


In [138]:
min({'Spades': 10, 'Clubs': 50, 'Hearts': 10, 'Diamonds': 30}.items(), key=lambda x: x[1])

('Spades', 10)